## Title
Data Raw Extraction 

### By:
Juan Gómez

### Date:
2025-05-16

### Description:

This notebook loads and joins data from the Yelp dataset to build the training structure.
It extracts data for user, reviews and business metadata to create a clean dataset for message classification.

# Data load

## Imports and configuration

In [1]:
from pathlib import Path

BASE_DIR = Path.cwd().resolve().parents[1]

In [2]:
import pandas as pd

In [3]:
pd.set_option("display.max_columns", None)

## Load dataset

### Business

In [4]:
# df_business = pd.read_json(BASE_DIR / "data/01_raw/yelp_academic_dataset_business.json", lines=True)

### User

In [5]:
# df_user = pd.concat(
#     [
#         chunk[chunk["review_count"] > 0]
#         for chunk in pd.read_json(
#             BASE_DIR / "data/01_raw/yelp_academic_dataset_user.json",
#             lines=True,
#             chunksize=100000,
#         )
#     ],
#     ignore_index=True,
# )

### Review

In [6]:
# df_review = pd.concat(
#     [
#         chunk[(chunk["text"].str.strip() != "") & (chunk["useful"] > 0)]
#         for chunk in pd.read_json(
#             BASE_DIR / "data/01_raw/yelp_academic_dataset_review.json",
#             lines=True,
#             chunksize=100000,
#         )
#     ],
#     ignore_index=True,
# )

In [7]:
# df_review2 = df_review.sort_values("date", ascending=False).head(1000100)

## Enrich reviews with user data

In [8]:
# df_review3 = df_review2.merge(df_user, on=["user_id"], how="left", suffixes=("", "_user"))

## Enrich reviews with business data

In [9]:
# df_review4 = df_review3.merge(df_business, on=["business_id"], how="left", suffixes=("", "_user"))

## Save reviews data raw

In [10]:
# df_review4.to_parquet(BASE_DIR / "data/02_intermediate/data_message_classifier_interm.parquet", index=False)

# Test Data Extract

In [11]:
import os

os.chdir("/Users/agomezj/Desktop/Juan-G/ml-message-classifier/")
print(os.getcwd())

/Users/agomezj/Desktop/Juan-G/ml-message-classifier


In [12]:
from src.pipelines.feature_pipeline.feature_pipeline import feature_pipeline

In [13]:
extract = feature_pipeline.named_steps["extract"].fit_transform(None)

2025-05-20 11:06:47.417 | INFO     | src.data.extract:_load_user:92 - Loading user data from /Users/agomezj/Desktop/Juan-G/ml-message-classifier/data/01_raw/yelp_academic_dataset_user.json
2025-05-20 11:07:09.743 | INFO     | src.data.extract:_load_review:111 - Loading review data from /Users/agomezj/Desktop/Juan-G/ml-message-classifier/data/01_raw/yelp_academic_dataset_review.json
2025-05-20 11:07:52.765 | INFO     | src.data.extract:_load_business:130 - Loading business data from /Users/agomezj/Desktop/Juan-G/ml-message-classifier/data/01_raw/yelp_academic_dataset_business.json
2025-05-20 11:07:58.721 | INFO     | src.data.extract:_merge_all:151 - Merging review, user, and business data
2025-05-20 11:08:55.716 | INFO     | src.data.extract:_save_if_needed:81 - Saving merged data to /Users/agomezj/Desktop/Juan-G/ml-message-classifier/data/01_raw/review_user_business_extract.parquet
